# Total Perspective Vortex
- Date: `14-07-2023 3:30PM`
- Third project in the AI branch of 42's system
- Project description: 
    - `Learn to use python tools to process large dimension data, and create a man-machine interface thanks to brainwaves.`
- Summary: 
    - `Brain computer interface with machine learning based on electoencephalographic data.`
    - This subject aims to create a brain computer interface based on electroencephalographic
data (EEG data) with the help of machine learning algorithms. Using a subject’s EEG
reading, you’ll have to infer what he or she is thinking about or doing - (motion) A or B
in a t0 to tn timeframe.
- Goals:
    - Process EEG datas (parsing and filtering)
    - Implement a dimensionality reduction algorithm
    - Use the pipeline object from scikit-learn
    - Classify a data stream in "real time"

  ##### Update 28th December 2023 - Continuing the ongoing work.
    - Set up environment.
    - Load data.
    - Filter out bad frequencies.
    - Analyze.
 
  ---


# EEG (Electroencephalography)

<img src="../assets/eeg-cover-intro.png" alt="Intro image" width="500">

---
## Steps:
V.1 Preprocessing, parsing and formating . <br>
V.2 Treatment pipeline . <br>
V.3 Implementation . <br>
V.4 Train, Validation and Test . <br>

---

#### we pick data related to hands vs feet
5, 9, 13 : Motor execution: hands vs feet (person executing the movement of his hands/feets for real)<br>
6, 10, 14 : Motor imagery: hands vs feet (person imagine the movement of his hands/feets)<br>
read: https://mne.tools/dev/generated/mne.datasets.eegbci.load_data.html

In [ ]:
import numpy as np
%matplotlib widget
import matplotlib.pyplot as plt
import mne
from mne import pick_types
from mne.channels import make_standard_montage
from mne.io import concatenate_raws, read_raw_edf
from mne.datasets import eegbci
from mne.preprocessing import ICA 

# DATA_SAMPLE_PATH = "../data/"
mne.set_log_level("CRITICAL")
mne.set_config('MNE_BROWSE_RAW_SIZE','20,8')
# CRITICAL ?? ---> check descripton bellow

- **DEBUG:** Detailed information, typically useful for debugging.
- **INFO:** General information about the program's progress.
- **WARNING:** Indicates a potential issue that doesn't prevent the program from running.
- **ERROR:** Indicates a more serious issue that may prevent certain functionality.
- **CRITICAL:** Indicates a critical error that might lead to the program's termination.

Setting the logging level to "CRITICAL" is often done when you want to minimize the amount of console output, especially when running scripts or programs in production environments where detailed logging may not be necessary.

If you want more detailed logging information during development or debugging, you might use a higher logging level such as "DEBUG" or "INFO."

### Data loading:

- The code loads EEG data for executing and imagining motor tasks for specified subjects and runs.<br>
- For each subject and run combination, EEG data is loaded, and events corresponding to rest, foot movement, and hand movement are annotated.<br>
- Annotations are created for both executing and imagining motor tasks, and the annotations are set to the respective raw data.<br>
- The processed raw data is appended to the raw_files list.<br>
- This code is designed for loading and preprocessing EEG data for motor task execution and imagery, making it ready for further analysis.<br>

#### code details:
- **`mne.events_from_annotations`**: extracts events from annotations in the raw EEG data.<br>
It's applied separately to executing and imagining motor tasks.<br>
The event_id parameter maps annotation labels to event codes.
- **`mne.annotations_from_events`**: creates annotations from events.<br>
It maps events to descriptions using the provided mapping.<br>
Annotations include information about event timing and descriptions.
    - ##### Parameters:
        - `events`: Event array.
        - `event_desc`: Dictionary mapping event codes to descriptions.
        - `sfreq`: Sampling Frequency:<br>
            - In the context of EEG data, the sfreq parameter stands for the "sampling frequency" of the raw data.<br>
            - Sampling frequency represents the number of samples (data points) obtained per unit of time, usually expressed in Hertz (Hz).<br>
            - For EEG data, the sampling frequency indicates how many samples are collected per second. Higher sampling frequencies provide more temporal detail but may require more storage and processing resources.<br>
            - In this code, raw_execution.info['sfreq'] retrieves the sampling frequency from the information associated with the raw_execution object.
        - `orig_time`: Original measurement time of the raw data.<br>
            - The orig_time parameter represents the original measurement time of the raw EEG data.<br>
            - It indicates the specific time point when the EEG recording began.<br>
            - The information about the original measurement time is important for maintaining the temporal context of the EEG data.<br>
            - In this code, raw_execution.info['meas_date'] retrieves the original measurement time from the information associated with the raw_execution object.<br>


In [ ]:
real_print=print
debug=True
def print(*arg, **kwargs):
	if debug:
		real_print(*arg, **kwargs)

In [ ]:
# EEG Data Loading and Annotation

# - T0 corresponds to rest
# - T1 corresponds to the onset of motion (real or imagined) of the left fist (in runs 3, 4, 7, 8, 11, and 12)
#   or both fists (in runs 5, 6, 9, 10, 13, and 14)
# - T2 corresponds to the onset of motion (real or imagined) of the right fist (in runs 3, 4, 7, 8, 11, and 12)
#   or both feet (in runs 5, 6, 9, 10, 13, and 14)

subject = [1]  # List of subject numbers (e.g., [1, 4])
run_execution = [5, 9, 13]  # Runs for executing motor tasks
run_imagery = [6, 10, 14]  # Runs for imagining motor tasks

raw_files = []

# Loop through each subject and associated runs for execution and imagery
debug=True
for person_number in subject:
    for i, j in zip(run_execution, run_imagery):
        # Load EEG data for executing motor tasks
        raw_files_execution = [read_raw_edf(f, preload=True, stim_channel='auto') for f in eegbci.load_data(person_number, i)]
        print(f"Raw files execution:{raw_files_execution}")
        raw_execution = concatenate_raws(raw_files_execution)
        print(f"Raw execution :{raw_execution}")

        # Load EEG data for imagining motor tasks
        raw_files_imagery = [read_raw_edf(f, preload=True, stim_channel='auto') for f in eegbci.load_data(person_number, j)]
        raw_imagery = concatenate_raws(raw_files_imagery)

        # Extract events and create annotations for executing motor tasks
        events, _ = mne.events_from_annotations(raw_execution, event_id=dict(T0=1, T1=2, T2=3))
        print(f"event :{events}")
        mapping = {1: 'rest', 2: 'do/feet', 3: 'do/hands'}
        annot_from_events = mne.annotations_from_events(
            events=events, event_desc=mapping, sfreq=raw_execution.info['sfreq'],
            orig_time=raw_execution.info['meas_date'])
        print(f"annot_from_events :{annot_from_events}")
        debug=False
        raw_execution.set_annotations(annot_from_events)

        # Extract events and create annotations for imagining motor tasks
        events, _ = mne.events_from_annotations(raw_imagery, event_id=dict(T0=1, T1=2, T2=3))
        mapping = {1: 'rest', 2: 'imagine/feet', 3: 'imagine/hands'}
        annot_from_events = mne.annotations_from_events(
            events=events, event_desc=mapping, sfreq=raw_imagery.info['sfreq'],
            orig_time=raw_imagery.info['meas_date'])
        raw_imagery.set_annotations(annot_from_events)

        # Append the processed raw data to the list
        raw_files.append(raw_execution)
        raw_files.append(raw_imagery)
print("Done")

In [ ]:
raw = concatenate_raws(raw_files)
originalRaw = raw.copy()
events, event_dict = mne.events_from_annotations(raw)
data = raw.get_data()

# display(raw.info)
print(len(events), event_dict)
# display(raw.ch_names)
# display(data)
raw.plot(scalings=dict(eeg=1e-4));

### filter any bad channels that were identified to have artifacts

In MNE-Python, the `pick_types` function is used to select channels based on their types. The function returns indices of channels that match the specified criteria. The `pick_types` function takes several parameters to filter channels based on their types.

- `raw.info`: This is the `info` attribute of the `raw` object, which contains information about the data, such as channel names, types, and other metadata.

- `meg=False`: This specifies that MEG channels should not be included in the selection. Setting it to `False` means excluding MEG channels.
  - MEG stands for Magnetoencephalography. MEG channels are sensors used in magnetoencephalography to measure the magnetic fields produced by neural activity in the brain. MEG is a non-invasive neuroimaging technique that complements other methods such as electroencephalography (EEG).

- `eeg=True`: This specifies that EEG channels should be included in the selection. Setting it to `True` means including EEG channels.

- `stim=False`: This specifies that stimulus channels should not be included in the selection. Setting it to `False` means excluding stimulus channels.
  - A stimulus channel is a specific type of channel in EEG recordings that is dedicated to marking the timing of presented stimuli during an experiment. This channel typically contains events or triggers that indicate when a stimulus was presented to the participant. These events are essential for aligning EEG data with specific experimental conditions, allowing researchers to analyze and interpret the brain's response to different stimuli accurately.
- `eog=False`: This specifies that EOG (electrooculogram) channels should not be included in the selection. Setting it to `False` means excluding EOG channels.
    - specefic electrods placed around the eyes, to capture the nerve activities
- `exclude='bads'`: This specifies that bad channels (channels marked as bad) should be excluded from the selection. Channels marked as bad are typically those with artifacts or other issues.

After executing this line, the `picks` variable will contain the indices of EEG channels (excluding bad channels) from the `raw` object. These indices can be used to access or manipulate EEG data from the specified channels in subsequent steps of your analysis.

In [ ]:
# filter any bad channels that were identified to have artifacts
picks = pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False, exclude='bads')
print(picks)

## Standard montages
**Standard Montage**: A standard montage defines the standardized positions of electrodes on the scalp. <br>
Different standard montages provided by MNE-Python:
- **'standard_1020'**: This is a standard EEG electrode montage with 21 electrodes based on the international 10-20 system.
- **'standard_1005'**: Another standard EEG electrode montage with 348 electrodes based on the extended 10-05 system.
- **'biosemi32'**: A BioSemi montage with 32 electrodes.
- **'biosemi64'**: A BioSemi montage with 64 electrodes.

**BioSemi** *is a company that specializes in the development and manufacturing of EEG (electroencephalogram) and other neurophysiological equipment. The BioSemi EEG system is a type of EEG recording system produced by BioSemi.*

In [ ]:
fig = make_standard_montage('biosemi64').plot();

Sets the EEG channel names and applies a standard EEG montage, which includes digitization points for the EEG electrodes.

This is a common and necessary step when working with EEG data in MNE-Python.

- **`eegbci.standardize(raw)`**: This function is used to standardize the channel names of the EEG data. It ensures that the channel names follow a consistent naming convention, making it easier to work with standard montages.

- **`make_standard_montage('standard_1005')`**: This function creates a standard EEG electrode montage based on the 10/05 system, which is a widely used electrode placement system. It defines the positions of EEG electrodes on the scalp.

- **`raw.set_montage(montage)`**: This line sets the created electrode montage (montage) to the EEG data in the raw object. This step aligns the electrode positions in the data with the standard positions defined by 'standard_1005', facilitating accurate spatial analysis.

In [ ]:
eegbci.standardize(raw)  # set channel names
biosemi_montage = make_standard_montage('standard_1005')
raw.set_montage(biosemi_montage)

---
## Power Spectral Density (PSD)
Plot the power spectral density (PSD) of the EEG data stored in the raw object. Here's what each line does:
Power Spectral Density (PSD) just descrives the characteristics of time signals (RMS on Frequency range))
- `raw.plot_psd(average=False)`: 
    - This line generates a plot of the power spectral density of the EEG data. 
    - The `average=False` argument means that it will plot the PSD separately for each channel rather than averaging across channels.

#### ressources:
- What is a power spectrum?: https://youtu.be/Gka11q5VfFI

In [ ]:
raw.plot_psd();
raw.plot_psd(average=True);

#### In a power spectral density (PSD) plot for EEG data, the two dotted vertical lines pointing to a range between 5 and 20 Hz typically indicate the frequency range of interest, often referred to as a specific frequency band.

The frequency range of interest in the context of EEG analysis typically includes the theta band (4-7 Hz), alpha band (8-13 Hz), beta band (13-30 Hz), and gamma band (30 Hz and above), with specific focus depending on the cognitive states or tasks under investigation.

- **Theta Band (4-7 Hz)**: The lower dotted line around 5 Hz might represent the lower bound of the theta frequency band, associated with drowsiness, relaxation, and light sleep.

- **Alpha Band (8-13 Hz)**: The upper dotted line around 20 Hz might represent the upper bound of the alpha frequency band, associated with a ***relaxed but awake state***. Alpha activity is often prominent when the ***eyes are closed***.

- **Beta Band (13-30 Hz)**: The beta frequency band in EEG typically falls in the range of approximately 13 to 30 Hz. In a power spectral density (PSD) plot, you would expect the beta band to be situated between the alpha and gamma bands. It's often associated with active cognitive ***processing, concentration, and alertness***.

- **Gamma Band (30-... Hz)** The gamma frequency range of approximately 30 Hz and above. Gamma oscillations are associated with ***high-frequency neural activity*** and are often linked to processes involving cognitive functions such as ***perception, attention, and memory encoding***.

These frequency bands are commonly examined in EEG analysis to study different cognitive states and physiological phenomena. 

The vertical lines serve as a visual guide to highlight specific frequency ranges of interest on the PSD plot. 

Researchers often focus on these bands to analyze brain activity related to different mental states or tasks.

---

## Notch Filter

**Notch Filter:** A specialized filter used in signal processing to selectively suppress or eliminate specific frequencies,

often applied in EEG data processing to remove interference from power line noise at the frequency of the electrical power grid (e.g., 50 Hz or 60 Hz).

In [ ]:
# Notch filter
notch_freq = 60
raw.notch_filter(notch_freq, fir_design='firwin')
raw.compute_psd().plot();

## Band-Pass filter to EEG data:

In [ ]:
# Band-pass filter keep only alpha and beta waves
low_cutoff = 8
high_cutoff = 30
raw.filter(low_cutoff, high_cutoff, fir_design='firwin')
raw.compute_psd().plot()
;

## Final result with basic filters

In [ ]:
originalRaw.compute_psd().plot()
raw.compute_psd().plot()
;
originalRaw.compute_psd().plot(average=True)
raw.compute_psd().plot(average=True)
;

## Events


1. `fig = mne.viz.plot_events(events, sfreq=raw.info['sfreq'], first_samp=raw.first_samp, event_id=event_dict)`: This line generates an event plot using MNE-Python's `plot_events` function. It visualizes the occurrence and types of events in the EEG data.

   - `events`: This is a NumPy array containing event information, typically obtained from an event file associated with the EEG recording.
   - `sfreq=raw.info['sfreq']`: Specifies the sampling frequency of the EEG data, which is extracted from the information stored in the `raw` object.
   - `first_samp=raw.first_samp`: Specifies the index of the first sample in the raw data. It ensures correct alignment of events with the EEG data.
   - `event_id=event_dict`: Specifies a dictionary mapping event labels to event codes. This helps in labeling different types of events in the plot.

2. `fig.subplots_adjust(right=0.7)`: This line adjusts the layout of the subplots in the figure, specifically expanding the space on the right side of the plot. It's often used to make room for legend annotations or additional information.

In summary, these lines create an event plot for EEG data, displaying the timing and types of events, and adjust the layout of the plot to accommodate additional information on the right side.

In [ ]:
print(":- events shape = ", events.shape)
print(":- raw.info['sfreq'] = ", raw.info['sfreq'])
print(":- first_samp = ", raw.first_samp)
print(":- event_id = ", event_dict)


In [ ]:
fig = mne.viz.plot_events(events, sfreq=raw.info['sfreq'], first_samp=raw.first_samp, event_id=event_dict)
# fig.subplots_adjust(right=0.7)
;

---

##  Independent Component Analysis (ICA) 
إستأصال الترددات المتداخلة مع الدماغ عن طريق التعلم العميق للالة المعد مسبفا 

##### perform Independent Component Analysis (ICA) on EEG data to identify and visualize independent components. 
##### It specifically focuses on identifying components related to eye movements (EOG artifacts) and prints the indices of the identified "bad" components. 
##### These components can then be excluded from further analysis or removed from the data to enhance the quality of EEG recordings.
- `rawClone = raw.copy()`: This line creates a copy of the raw EEG data. The raw object is typically an instance of the mne.io.Raw class representing continuous EEG data. Creating a copy (rawClone) is a common practice to avoid modifying the original data during processing.
- `ica = ICA(n_components=20, method='fastica', random_state=97)`: This line initializes an Independent Component Analysis (ICA) object. ICA is a technique used for blind source separation and is commonly applied to EEG data to identify and remove artifacts. The parameters specified include:
    - `n_components=20`: Specifies the number of independent components to estimate. In this case, it's set to 20.
      - explanation:
        - indicates that you are requesting the ICA algorithm to identify and separate the EEG data into 20 independent components. Each independent component represents a linear combination of the original EEG channels, capturing different sources of neural or non-neural activity.
        - Choosing the appropriate number of components is a crucial step in ICA. If the number is too low, you may not capture all relevant sources in the data. If it's too high, you might end up with components that represent noise or overfit the data.
        - In practice, determining the optimal number of components often involves exploring the data and using domain knowledge or statistical criteria to guide the selection. Researchers might assess the explained variance, examine the topographic maps of the components, or use other metrics to guide the choice of n_components based on the characteristics of the EEG data and the goals of the analysis.

    - `method='fastica'`: Specifies the ICA algorithm to use. 'fastica' is a commonly used algorithm for EEG artifact removal.
    - `random_state=97`: Sets the random seed for reproducibility. The same seed ensures that the results are consistent across runs.
- `ica.fit(rawClone, picks=picks)`: This line fits the ICA model to the EEG data (rawClone). The picks parameter is used to specify the subset of channels on which to perform the ICA. It's a good practice to limit the ICA to relevant channels, often excluding certain channels like EOG (electrooculogram) or other non-EEG channels.

- `ica.plot_components()`: This line generates a plot of the independent components estimated by the ICA. The purpose of this plot is to visually inspect the components and identify those that correspond to artifacts (such as eye blinks, muscle activity) as opposed to neural sources.

- `eog_indicies, scores= ica.find_bads_eog(raw, ch_name='Fpz', threshold=1.5)`: This line identifies "bad" independent components that likely represent artifacts related to eye movements. It uses the find_bads_eog method, which leverages information about the EOG channels ('Fpz' in this case) to identify components associated with eye movements. The threshold parameter determines the sensitivity of the algorithm.

- `print("Bad indicies: " + str(eog_indicies))`: This line prints the indices of the identified "bad" independent components. These components can be further inspected and potentially removed from the EEG data to improve data quality.

In [ ]:
raw_corrected = raw.copy()
n_components = 20

ica = ICA(n_components=20, method='fastica', fit_params=None, random_state=97)

ica.fit(raw_corrected, picks=picks)

ica.plot_components()

eog_indicies, scores= ica.find_bads_eog(raw, ch_name='Fpz', threshold=1.5)
print("eog_indicies: ", eog_indicies)
ica.plot_scores(scores, exclude=eog_indicies)
ica.exclude.extend(eog_indicies)
raw_corrected = ica.apply(raw_corrected, n_pca_components = n_components, exclude = ica.exclude)
;

In [ ]:
ica.plot_sources(raw)
;

## Before / After all filters:

In [ ]:
originalRaw.plot(scalings=dict(eeg=1e-4))
raw.plot(scalings=dict(eeg=1e-4))
;

## Plot 30 channel output during 100 seconds before and after the filter

In [ ]:
print("Number of channels:", len(raw.ch_names))
print("Total duration:", raw.times[-1], "seconds")
originalRaw.plot(n_channels=30, start=0, duration=100, scalings=dict(eeg=250e-6))
raw.plot(n_channels=30, start=0, duration=100, scalings=dict(eeg=200e-6))
;

## Before and After:
### demo with 3 channels only, for clear plotting

In [ ]:
# originalRaw.plot(n_channels=3, scalings=dict(eeg=250e-6))
originalRaw.plot(n_channels=3, start=0, duration=40, scalings=dict(eeg=250e-6))
raw.plot(n_channels=3, start=0, duration=40, scalings=dict(eeg=250e-6))

# raw.plot(n_channels=5, start=0, duration=40, scalings=dict(eeg=250e-6))
;

## Share Data with other notebooks

In [ ]:
!pip install pickleshare
# Notebook 1
from pickleshare import PickleShareDB
db = PickleShareDB('./tmp')
db['raw'] = raw
db['originalRaw'] = originalRaw
db['picks'] = picks
db['events'] = events
db['evens'] = events
db['events_ids'] = event_dict